In [1]:
lib = %pwd
lib

'd:\\practice\\MLproject\\research'

In [2]:
import os

file_dir,file_name = os.path.split(lib)
file_dir

'd:\\practice\\MLproject'

In [3]:
import sys

sys.path.append(file_dir)

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    objective: str
    colsample_bytree:  float
    learning_rate:  float
    max_depth:  int 
    alpha:  int 
    n_estimators:  int
    target_column: str

In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [7]:
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = r"D:\practice\MLproject\config\config.yaml",
        params_filepath = r"D:\practice\MLproject\params.yaml",
        schema_filepath = r"D:\practice\MLproject\schema.yaml"):

        config_filepath = Path(config_filepath)
        params_filepath = Path(params_filepath)
        schema_filepath = Path(schema_filepath)

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            objective = params.objective,
            colsample_bytree = params.colsample_bytree,
            learning_rate = params.learning_rate,
            max_depth = params.max_depth,
            alpha = params.alpha,
            n_estimators = params.n_estimators,
            target_column = schema.name
            
        )

        return model_trainer_config

In [9]:
import pandas as pd
import os
from src.mlProject import logger
import xgboost as xgb
import joblib

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        
        xg_reg = xgb.XGBRegressor(objective =self.config.objective, 
                          colsample_bytree = self.config.colsample_bytree, 
                          learning_rate = self.config.learning_rate,
                          max_depth = self.config.max_depth, 
                          alpha = self.config.alpha, 
                          n_estimators = self.config.n_estimators)

        xg_reg.fit(train_x, train_y)
        
        joblib.dump(xg_reg, os.path.join(self.config.root_dir, self.config.model_name))
        return os.path.join(self.config.root_dir, self.config.model_name)



In [13]:
STAGE_NAME = "Model Trainer stage"

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_training = ModelTrainer(config=model_trainer_config)
    model_training.train()
    logger.info(f"Model trained successfully and saved at: {model_training.train()}")
except Exception as e:
    logger.exception(f"Error occurred during {STAGE_NAME} execution: {str(e)}")
    raise e

[2024-04-21 21:41:32,215: INFO: common: yaml file: D:\practice\MLproject\config\config.yaml loaded successfully]
[2024-04-21 21:41:32,223: INFO: common: yaml file: D:\practice\MLproject\params.yaml loaded successfully]
[2024-04-21 21:41:32,232: INFO: common: yaml file: D:\practice\MLproject\schema.yaml loaded successfully]
[2024-04-21 21:41:32,235: INFO: common: created directory at: artifacts]
[2024-04-21 21:41:32,235: INFO: common: created directory at: artifacts/model_trainer]


[2024-04-21 21:41:32,663: INFO: 1990927345: Model trained successfully and saved at: artifacts/model_trainer\model.joblib]
